# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project 2: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: **

Identifying whether or not a student might need early intervention is a binary classification problem. Either a student needs early intervention, or a student does not. Even if the students' course performance is put on a continuous scale, there would still have to be a decision threshold that classifies a student in a binary way. And in this particular supervised learning application, we are going to train the learner on whether or not a studen with particular attribute values passed their final exam and graduated high school.

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [24]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [25]:
# TODO: Calculate number of students
n_students = len(student_data.index)

# TODO: Calculate number of features
n_features = len(student_data.columns[:-1])

# TODO: Calculate passing students
n_passed = sum(student_data['passed']=='yes')

# TODO: Calculate failing students
n_failed = sum(student_data['passed']=='no')

# TODO: Calculate graduation rate
grad_rate = 100*np.float(n_passed)/np.float(n_students)

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [26]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [27]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [42]:
# TODO: Import any additional functionality you may need here
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import Normalizer

normer = Normalizer()

# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the dataset into the number of training and testing points above
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, stratify=y_all, test_size=num_test, random_state=42)

X_train = normer.fit_transform(X_train)
X_test = normer.transform(X_test)

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

print "Grad rate of the training set: {:.2f}%".format(100 * (y_train == 'yes').mean())
print "Grad rate of the testing set: {:.2f}%".format(100 * (y_test == 'yes').mean())

Training set has 300 samples.
Testing set has 95 samples.
Grad rate of the training set: 67.00%
Grad rate of the testing set: 67.37%


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. What are the general applications of each model? What are their strengths and weaknesses? Given what you know about the data, why did you choose these models to be applied?*

**Answer: **

1.) Decision Tree: http://scikit-learn.org/stable/modules/tree.html

The ID3 Decision Tree Classifier takes a training data set with attribute-classification pairings, and splits the data along the attribute that best separates the data into positive and negative classifications. For data sets with many attributes, the algorithm iterates this task until the positive and negative classifications are perfectly separated, or until there are no more attributes with which to split the data.


Industry Application of Decision Trees: 

Stock Trading 
Wu, M. C., Lin, S. Y., & Lin, C. H. (2006). An effective application of decision tree to stock trading. Expert Systems with Applications, 31(2), 270-274.

Advertisting
Kim, J. W., Lee, B. H., Shaw, M. J., Chang, H. L., & Nelson, M. (2001). Application of decision-tree induction techniques to personalized advertisements on internet storefronts. International Journal of Electronic Commerce, 5(3), 45-62.


An advantage to this algorithm is in its relatively low computational requirements per iteration (being linear relative to the number of attributes and number of instances). However, because the algorithm is set to continue splitting either until all the instances are classified, or until there are no more attributes with which to split the data, it is prone to overfitting on the training data at the expense of accuracy in the testing set. It is possible to avoid this type of "over-splitting" by setting a hard cut-off for the tree depth (how many levels of nodes the tree can split to), but this approach may also lead to poor testing accuracy if the data is noisy (that is, even the attributes with the best information gain still have a mix of positive and negative instances). 

So with the above in mind, the Decision Tree Classifier may be a good candidate for this task if the data has low noise, and there are attributes that "cleanly" split the positive and negative instances.

2.) Support Vector Classifier: http://scikit-learn.org/stable/modules/svm.html

The Support Vector Classifier (SVC), also takes a set of attribute-classification pairings and separates the positive and negative instances. But unlike the Decision Tree Classifier, which splits attributes to separate positive and negative instances, the Support Vector Machine uses a kernel function - in this case, the Radial Basis Function (RBF) - to "project" a multi-dimensional attribute set into a space where the features can be linearly separated so that the margin (the space between the boundary and the closest positive and negative instances) can be maximized. 

Industry Applications of Support Vector Machines:

Facial Recognition
Osuna, E., Freund, R., & Girosi, F. (1997, June). Training support vector machines: an application to face detection. In Computer vision and pattern recognition, 1997. Proceedings., 1997 IEEE computer society conference on (pp. 130-136). IEEE.

Financial Time Series Forecasting:
Tay, F. E., & Cao, L. (2001). Application of support vector machines in financial time series forecasting. Omega, 29(4), 309-317.

The Radial Basis Function can also be tuned on two important parameters - the 'C' value, and the 'gamma' value. The 'C' parameter controls the trade-off between accurately classifying all instances, and the simplicity of the decision boundary. A higher C means a more complex boundary, which may capture small differences in the attributes of borderline cases, but may also overfit the training data in an attempt to correctly classify as many training instances as possible. The gamma value can be thought of, according to sklearn, as the inverse of the radius of influence of a given sample. A higher gamma value means that any given vector only has a minimal radius of influence as to how subsequent instances will be classified. And thus, with a big enough gamma, the entire training set would be used as the support vector in drawing the boundaries. This may prevent a model from over-generalizing a small dataset, but if a population has higher variance than the training sample, then the high-gamma RBF will not properly account for those instances not in the small circle of influence of our given data.

Because the Radial Basis Function is tuneable on the C and gamma parameters, the SVC with an RBF kernel has high upside in that it can work with complicated data to draw "clusters" and point out groups of "at-risk" students with similar attribute values. But, because of the complexity involved in applying the RBF kernel to our many attributes, the SVC runs the risk of being very computationally expensive. It is likely to work well in our small training set of 300 samples, but it is unlikely to do so for larger datasets (i.e. state-wide data, region-wide data). 

3.) AdaBoost (Decision Tree Classifier): http://scikit-learn.org/stable/modules/ensemble.html#adaboost

The AdaBoost learner takes a "weak" learner (in this case, the Decision Tree Classifier), and re-trains it using an iterative weighting method. If the weak learner misclassifies an instance, the probability of that instance re-appearing is increased. If the instance is classified correctly, the probability of appearance is decreased, which allows the weak leaner to re-train on cases where it was wrong. At the end of the boosted training, the learner to be used for the testing set is the weighted "vote" (or sum) of all the weak learners. 

Industry Applications of AdaBoost:

Facial Recognition
Viola, P., & Jones, M. J. (2004). Robust real-time face detection. International journal of computer vision, 57(2), 137-154.

Speech Recognition:
Schwenk, H. (1999, March). Using boosting to improve a hybrid HMM/neural network speech recognizer. In Acoustics, Speech, and Signal Processing, 1999. Proceedings., 1999 IEEE International Conference on (Vol. 2, pp. 1009-1012). IEEE.


This is likely to give the boosted learner an advantage in prediction accuracy over the non-boosted learner because of its ability to self-correct. However, because AdaBoost is iterative in nature, we no longer have the advantage of small computational cost that the single decision tree had. But, because we still have a relatively small dataset (300 training points), if the boosted learner shows to have a significantly larger testing accuracy, it may be worth the higher computational cost. 

In this analysis, the AdaBoost method is a good choice because it gives us a chance to compare the results of an ensemble method, side-by-side with its underlying weak learner. The AdaBoost method, like its weak learner, will work well if the dataset shows that the positive and negative instances can be cleanly split by a few "key" attributes. As well, this method will work best if there are not many "outliers" on either side that could influence the way the boosted tree is drawn.

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [29]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [43]:
# TODO: Import the three supervised learning models from sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier

# TODO: Initialize the three models
clf_A = DecisionTreeClassifier(random_state=42)
clf_B = SVC(random_state=42)
clf_C = AdaBoostClassifier(random_state=42)

# TODO: Set up the training set sizes
X_train_100 = X_train[:100]
y_train_100 = y_train[:100]

X_train_200 = X_train[:200]
y_train_200 = y_train[:200]

X_train_300 = X_train
y_train_300 = y_train

# TODO: Execute the 'train_predict' function for each classifier and each training set size
train_predict(clf_A, X_train_100, y_train_100, X_test, y_test)
train_predict(clf_A, X_train_200, y_train_200, X_test, y_test)
train_predict(clf_A, X_train_300, y_train_300, X_test, y_test)

train_predict(clf_B, X_train_100, y_train_100, X_test, y_test)
train_predict(clf_B, X_train_200, y_train_200, X_test, y_test)
train_predict(clf_B, X_train_300, y_train_300, X_test, y_test)

train_predict(clf_C, X_train_100, y_train_100, X_test, y_test)
train_predict(clf_C, X_train_200, y_train_200, X_test, y_test)
train_predict(clf_C, X_train_300, y_train_300, X_test, y_test)


Training a DecisionTreeClassifier using a training set size of 100. . .
Trained model in 0.0040 seconds
Made predictions in 0.0000 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0000 seconds.
F1 score for test set: 0.5785.
Training a DecisionTreeClassifier using a training set size of 200. . .
Trained model in 0.0080 seconds
Made predictions in 0.0000 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0000 seconds.
F1 score for test set: 0.6613.
Training a DecisionTreeClassifier using a training set size of 300. . .
Trained model in 0.0130 seconds
Made predictions in 0.0000 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0000 seconds.
F1 score for test set: 0.7132.
Training a SVC using a training set size of 100. . .
Trained model in 0.0020 seconds
Made predictions in 0.0010 seconds.
F1 score for training set: 0.7952.
Made predictions in 0.0010 seconds.
F1 score for test set: 0.8050.
Training a SVC using a training set size of 200. . 

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - Decision Tree Classifier**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |        0.0040           |       0.0000           | 1.0000           |    0.5785       |
| 200               |        0.0070           |       0.0000           | 1.0000           |    0.6613       |
| 300               |        0.0130           |       0.0000           | 1.0000           |    0.7132       |

** Classifer 2 - Support Vector Classifier**

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |     0.0020              |    0.0010              |     0.7952       |    0.8050       |
| 200               |     0.0060              |    0.0020              |     0.7879       |    0.8050       |
| 300               |     0.0130              |    0.0020              |     0.8024       |    0.8050       |

** Classifer 3 - AdaBoost Classifier (Base Estimator: Decision Tree Classifier)**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |     0.2070              |        0.0100          |     1.0000       |    0.6970       |
| 200               |     0.1730              |        0.0100          |     0.9736       |    0.7031       |
| 300               |     0.1870              |        0.0100          |     0.9440       |    0.7407       |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Chosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: **
The best classifier, as a combination of prediction performance (as indexed by the test F1 score), as well as cost of computation (as indexed by training and prediction time) is the support vector classifier (SVC) with an average F1 testing score of 0.8050 and an average training time of 0.007s. Although the Decision Tree Classifier best fits the training data, with a 1.0 average F1 training score, this is just a symptom of overfitting, as its testing F1 score is not nearly as good (avg testing F1: 0.6510). The AdaBoost Classifier (with base estimator=Decision Tree Classifier) does notably better than its weak learner (average testing F1 score: 0.7136), but it comes at the cost of computational resources (average training time: 0.1890), and still has a lower average F1 score than the SVC.

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. For example if you've chosen to use a decision tree or a support vector machine, how does the model go about making a prediction?*

**Answer: **
The Support Vector Classifier (SVC) makes its prediction by taking in many groups of student attributes, known as the "training vectors", as well as what classification group they are a part of (in this case, whether or not the student passed their final exam and graduated school). Then, the SVC draws a boundary between the two groups in a way to maximize the distance between the border and the closest borderline students. 

Sometimes, it is easy to draw the boundary between two groups of students. It could be as simple as drawing a straight line between two separate groups

![](https://udacity-github-sync-content.s3.amazonaws.com/_imgs/8650/1461641602/svmFeatureSpace.gif)

Unfortunately, many datasets are more complex, and cannot be separated by a straight line. However, the support vector classifier may still be able to separate distinct groups with similar features using the "kernel trick"

![](https://udacity-github-sync-content.s3.amazonaws.com/_imgs/19273/1464624531/1gvce.png)

By employing a kernel function, the support vector classifier in effect changes the "lens" through which we look at our data, and makes it so that we can once again separate our data in a linear way. And even in a dataset where we have many (>3) dimensions of input attributes, the kernel can still "project" the data into a space where the classifier can draw a straight line between groups.

When the model is tested against new students without knowing whether or not they will pass their final exams, the SVC will use their other, known attributes to group them with other students having similar attributes. The border-line cases will be grouped based on which side of the boundary they fall.

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [45]:
# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer


# TODO: Create the parameters list you wish to tune
C_range = np.logspace(-2, 8, 11)
gamma_range = np.logspace(-7, 3, 11)
kernels=['linear','poly','rbf']
degrees=[2,3,4]
parameters = dict(gamma=gamma_range, C=C_range)

# TODO: Initialize the classifier
clf = SVC(random_state=42)

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score,pos_label="yes")

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(estimator=clf,scoring=f1_scorer,param_grid=parameters)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj=grid_obj.fit(X_train,y_train)

# Get the estimator
clf = grid_obj.best_estimator_

print clf_B

print clf

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=42, shrinking=True,
  tol=0.001, verbose=False)
SVC(C=100.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.10000000000000001,
  kernel='rbf', max_iter=-1, probability=False, random_state=42,
  shrinking=True, tol=0.001, verbose=False)
Made predictions in 0.0000 seconds.
Tuned model has a training F1 score of 0.8330.
Made predictions in 0.0150 seconds.
Tuned model has a testing F1 score of 0.7945.


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: **

Untuned Model: C=1.0, gamma='auto', kernel='rbf', random state=42

Training Size: 300, Training F1: 0.8024, Testing F1: 0.8050

Tuned Model: C=100.0, gamma='0.10000000000000001', kernel='rbf', random state=42

Training Size: 300, Training F1: 0.8330, Testing F1: 0.7945

The tuned model did better on the training set, but did slightly worse on the testing set. Both models trained and tested on normalized inputs.

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.